In [1]:
!pip install tsfresh

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, cv, Pool, CatBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tsfresh import extract_features
from tqdm.auto import tqdm, trange
from sklearn.linear_model import RidgeClassifier, LogisticRegression, LogisticRegressionCV
import random

In [3]:
data = pd.read_csv('dataset.csv',parse_dates=['client_start_date','partnerrolestart_date'])
simple_sub = pd.read_csv('submission_example.csv')

In [4]:
def get_labels(data):
    date = pd.to_datetime('2020-9-01')
    labels = data.groupby('clientbankpartner_pin')[['client_start_date']].agg('max')
    labels['score'] = labels['client_start_date'].apply(lambda x: 0 if x >= date else 1)
    labels['label_count'] = data.groupby('clientbankpartner_pin')['client_start_date'].agg(lambda x: len([i for i in x.tolist() if i >= date]) )
    return labels

labels = get_labels(data)

In [5]:
train_data = data.copy()
train_data = train_data[train_data['client_start_date'] < pd.to_datetime('2020-9-01')]

In [6]:
def distance_calc(x):
    dist = []
    for i in range(len(x)-1):
        dist += [x[i+1] - x[i]]
    return dist

def create_time_features(df,report_date = '2020-12-01'):
    df['month'] = df['client_start_date'].dt.month
    df['day'] = df['client_start_date'].dt.day
    df['week'] = df['client_start_date'].dt.day.map(lambda x: x // 7)
    df['minute'] = df['client_start_date'].dt.minute
    df['year'] = df['client_start_date'].dt.year

    df['dayofweek'] = df['client_start_date'].dt.dayofweek
    df['all_year_time'] = df['client_start_date'].apply(lambda x: (x.year - 2019) * 365 + x.month * 30 + x.day)
    df['cl_is_weekend'] = (df['dayofweek'] > 4).astype('int16')
    
    df['time_to_report'] = pd.to_datetime(report_date) - df['client_start_date']
    df['days_to_report'] = df['time_to_report'].dt.days.astype(int)
    
    return df

def get_time_features(df):
    time_day_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(['mean', 'std', 'min', 'max', 'median','count'])
    time_day_features.columns = [f'time_day_features_{c}' for c in time_day_features.columns]
    time_day_features['time_day_features_diff'] = time_day_features['time_day_features_max'] - time_day_features['time_day_features_min']
    time_day_features['time_day_features_diversity'] = time_day_features['time_day_features_diff'] / time_day_features['time_day_features_count']
    
    diff_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(lambda x: distance_calc(x.tolist()))
    time_day_features['max_diff'] = diff_features.apply(lambda x: max(x) if x != []  else -100)
    time_day_features['min_diff'] = diff_features.apply(lambda x: min(x) if x != []  else -100)
    time_day_features['mean_diff'] = diff_features.apply(lambda x: np.mean(x) if x != []  else -100)
    time_day_features['std_diff'] = diff_features.apply(lambda x: np.std(x) if x != []  else -100)
    time_day_features['hard_time_step_polyfit'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[1] if len(x) > 2  else -100)
    time_day_features['hard_time_step_polyfit_st'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 2  else -100)

    return time_day_features

def get_src_code_features(df):
        src = df.groupby('clientbankpartner_pin')['partner_src_type_ccode'].agg(['first'])     
        return src


def partnerrolestart_date_feature(df,report_date = '2020-12-01'):
    df['days_from_start_to_report'] = pd.to_datetime(report_date) - df['partnerrolestart_date']
    start_day_features = df.groupby('clientbankpartner_pin')['days_from_start_to_report'].agg(['min'])
    start_day_features['days_from_start_to_report'] = start_day_features['min'].dt.days
    return start_day_features.drop(['min'],axis=1)

def get_user_features(df):
    user_strs = df.groupby('clientbankpartner_pin')['client_pin'].agg(lambda x:x.tolist())
    user_features = pd.DataFrame()
    user_features['user_nunique'] = user_strs.apply(lambda x: len(np.unique(x)))
    user_features['user_diversity'] = user_features['user_nunique'] / user_strs.apply(len)
    
    user_data = df.groupby('client_pin')['clientbankpartner_pin'].agg(['count'])
    user_data['client_pin'] = user_data.index
    user_data = user_data.rename({'count':'count_tr_per_user'},axis=1)
    user_data.index = range(len(user_data))
    df = df.merge(user_data)
    
    user_features['user_max_act'] =  df.groupby('clientbankpartner_pin')['count_tr_per_user'].agg('max')
    user_features['user_mean_act'] =  df.groupby('clientbankpartner_pin')['count_tr_per_user'].agg('mean')
    return user_features


def last_count(df,bins=[5,10,15,20,25,30,35,45,50,60]):
    feature_df = pd.DataFrame(index=df.groupby('clientbankpartner_pin')['client_pin'].agg('count').index)
    for x in bins:
        feature_df[f'count_{x}_prev'] = df[df['days_to_report'] > x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'count_{x}_last'] = df[df['days_to_report'] <= x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'percent_{x}'] =  feature_df[f'count_{x}_last'] / (feature_df[f'count_{x}_last'] + feature_df[f'count_{x}_prev'])
        
    return feature_df.fillna(-1)

def create_tsfresh_features(df):
    df_features = extract_features(train_data,column_id='clientbankpartner_pin',column_value='days_to_report')
    return df_features

def compute_all_features(data,report_date):
    data = create_time_features(data,report_date)
    time_features = get_time_features(data)
    src_features = get_src_code_features(data)
    start_day_features = partnerrolestart_date_feature(data,report_date)
    user_features = get_user_features(data)
    last_count_feats = last_count(data)
    
    all_data = pd.concat([time_features,
                      src_features,
                      start_day_features,
                      last_count_feats,
                      user_features,
                      ],axis=1)
    return all_data

In [7]:
train_feats = compute_all_features(train_data,'2020-9-01')

In [8]:
#train_tsmix_feats = extract_features(train_data,column_id='clientbankpartner_pin',column_value='days_to_report')
#train_tsmix_feats.to_parquet('train_tsmixer.parquet')
train_tsmix_feats = pd.read_parquet('train_tsmixer.parquet')

In [9]:
all_train_data = pd.concat([train_feats.sort_index(),
                            train_tsmix_feats.sort_index(),
                            labels.loc[train_feats.index]],axis=1)

In [10]:
all_train_data = all_train_data.T.drop_duplicates().T

In [11]:
def get_val_splits(data,num_splits=10,n_val=1000,random_seed=56):
    date = pd.to_datetime('2020-6-01')
    labels = data.groupby('clientbankpartner_pin')[['client_start_date']].agg('max')
    good_partners = labels[labels['client_start_date'] >= date].index.tolist()
    all_partners = labels.index.tolist()
    train_idxs, val_idxs = [],[]
    for i in range(num_splits):
        random.seed(random_seed+i)
        random.shuffle(good_partners)
        val_part = good_partners[:n_val]
        tr_part = [part for part in all_partners if part not in val_part]
        train_idxs.append(tr_part)
        val_idxs.append(val_part)
    return train_idxs, val_idxs

train_splits, eval_splits = get_val_splits(train_data)

In [12]:
params = {'iterations':1_000,
          'loss_function':'CrossEntropy',
          'learning_rate':0.01,
          'task_type':'CPU',
          'min_child_samples':20,
          'max_depth':5,
          'eval_metric':'AUC',
          'random_seed':56}

drop_cols = ['label_count','score','client_start_date']

In [13]:
models = []
scores = []
for i in trange(len(train_splits)):
    train_pool = Pool(data=all_train_data.loc[train_splits[i]].drop(drop_cols,axis=1),
                      label=all_train_data.loc[train_splits[i]]['score'],
                      cat_features=['first'])
    eval_pool = Pool(data=all_train_data.loc[eval_splits[i]].drop(drop_cols,axis=1),
                      label=all_train_data.loc[eval_splits[i]]['score'],
                      cat_features=['first'])
    
    cbm = CatBoostClassifier(**params)
    cbm.fit(train_pool,eval_set=eval_pool,verbose=300)
    scores.append(cbm.get_best_score())
    models.append(cbm)

  0%|          | 0/10 [00:00<?, ?it/s]

0:	test: 0.6806949	best: 0.6806949 (0)	total: 154ms	remaining: 2m 33s
300:	test: 0.7526004	best: 0.7526004 (300)	total: 21.7s	remaining: 50.4s
600:	test: 0.7565302	best: 0.7565557 (599)	total: 41s	remaining: 27.2s
900:	test: 0.7575754	best: 0.7576095 (896)	total: 1m	remaining: 6.59s
999:	test: 0.7575967	best: 0.7576734 (904)	total: 1m 6s	remaining: 0us

bestTest = 0.7576733611
bestIteration = 904

Shrink model to first 905 iterations.
0:	test: 0.7095159	best: 0.7095159 (0)	total: 93.2ms	remaining: 1m 33s
300:	test: 0.7941707	best: 0.7942414 (298)	total: 20.4s	remaining: 47.3s
600:	test: 0.7980487	best: 0.7980612 (593)	total: 40s	remaining: 26.6s
900:	test: 0.7985191	best: 0.7988106 (828)	total: 59s	remaining: 6.48s
999:	test: 0.7990541	best: 0.7991124 (990)	total: 1m 5s	remaining: 0us

bestTest = 0.7991124026
bestIteration = 990

Shrink model to first 991 iterations.
0:	test: 0.7219215	best: 0.7219215 (0)	total: 90.4ms	remaining: 1m 30s
300:	test: 0.7687169	best: 0.7687754 (299)	total:

In [14]:
np.mean([x['validation']['AUC'] for x in scores])

0.7762600665718308

In [15]:
imp = models[0].get_feature_importance(prettified=True).set_index('Feature Id')
for i in range(1,10):
    imp += models[i].get_feature_importance(prettified=True).set_index('Feature Id')

In [16]:
best_features = imp.sort_values(by='Importances')[::-1][:128].index.tolist()

In [17]:
imp.sort_values(by='Importances')[::-1].head(30)

Importances
Feature Id                                                     
time_day_features_min                                136.701402
count_60_last                                         64.270272
count_50_last                                         45.629589
count_45_last                                         43.303023
days_to_report__quantile__q_0.1                       35.056781
days_to_report__quantile__q_0.2                       27.810390
count_35_last                                         23.305033
count_25_last                                         20.418460
days_to_report__variation_coefficient                 19.890940
days_to_report__quantile__q_0.3                       19.453420
count_30_last                                         15.239982
days_to_report__quantile__q_0.4                       14.731559
time_day_features_mean                                11.419219
count_20_last                                         10.185212
time_day_features_median                               9.507352
days_to_report__root_mean_square                       8.874170
count_15_last                                          7.735365
days_to_report__quantile__q_0.6                        7.602162
days_to_report__index_mass_quantile__q_0.8             5.939308
days_to_report__quantile__q_0.7                        4.770859
days_to_report__quantile__q_0.8                        4.605820
days_to_report__benford_correlation                    4.579878
time_day_features_diversity                            3.995105
days_to_report__skewness                               3.667470
hard_time_step_polyfit_st                              3.623173
first                                                  3.454108
days_to_report__agg_linear_trend__attr_"rvalue"...     3.441503
days_to_report__change_quantiles__f_agg_"mean"_...     3.347632
days_to_report__mean_abs_change                        3.267467
days_to_report__cwt_coefficients__coeff_0__w_2_...     3.173292

In [18]:
params = {'iterations':2_000,
          'loss_function':'CrossEntropy',
          'learning_rate':0.01,
          'task_type':'CPU',
          'min_child_samples':20,
          'max_depth':4,
          'eval_metric':'AUC',
          'random_seed':56}

In [19]:
models = []
scores = []
for i in trange(len(train_splits)):
    train_pool = Pool(data=all_train_data.loc[train_splits[i]][best_features],
                      label=all_train_data.loc[train_splits[i]]['score'],
                      cat_features=['first'])
    eval_pool = Pool(data=all_train_data.loc[eval_splits[i]][best_features],
                      label=all_train_data.loc[eval_splits[i]]['score'],
                      cat_features=['first'])
    
    cbm = CatBoostClassifier(**params)
    cbm.fit(train_pool,eval_set=eval_pool,verbose=300)
    scores.append(cbm.get_best_score())
    models.append(cbm) 

  0%|          | 0/10 [00:00<?, ?it/s]

0:	test: 0.6692993	best: 0.6692993 (0)	total: 15.8ms	remaining: 31.7s
300:	test: 0.7548399	best: 0.7548399 (300)	total: 4.18s	remaining: 23.6s
600:	test: 0.7586505	best: 0.7586633 (599)	total: 8.22s	remaining: 19.1s
900:	test: 0.7601641	best: 0.7602407 (899)	total: 12.3s	remaining: 15s
1200:	test: 0.7608921	best: 0.7608964 (1199)	total: 16.2s	remaining: 10.8s
1500:	test: 0.7619012	best: 0.7619012 (1500)	total: 20.2s	remaining: 6.71s
1800:	test: 0.7619544	best: 0.7622440 (1773)	total: 24.3s	remaining: 2.68s
1999:	test: 0.7617714	best: 0.7623547 (1856)	total: 26.9s	remaining: 0us

bestTest = 0.7623546543
bestIteration = 1856

Shrink model to first 1857 iterations.
0:	test: 0.7257899	best: 0.7257899 (0)	total: 14.1ms	remaining: 28.1s
300:	test: 0.7943851	best: 0.7944059 (298)	total: 4.19s	remaining: 23.6s
600:	test: 0.7980071	best: 0.7981528 (591)	total: 8.23s	remaining: 19.2s
900:	test: 0.7997140	best: 0.7998639 (861)	total: 12.2s	remaining: 14.9s
1200:	test: 0.7991603	best: 0.8002760 (9

In [20]:
np.mean([x['validation']['AUC'] for x in scores])

0.7794836222555394

In [21]:
test_data = data.copy()
test_feats = compute_all_features(test_data,'2020-12-01')

In [22]:
#test_tsmix_feats = extract_features(test_data,column_id='clientbankpartner_pin',column_value='days_to_report')
#test_tsmix_feats.to_parquet('test_tsmixer.parquet')
test_tsmix_feats = pd.read_parquet('/test_tsmixer.parquet')

In [24]:
all_test_data = pd.concat([test_feats.sort_index(),test_tsmix_feats.sort_index()],axis=1)

In [25]:
all_test_data_lc = all_test_data.loc[simple_sub['clientbankpartner_pin']]

In [26]:
test_pool = Pool(all_test_data_lc[best_features],
                 cat_features=['first'])

In [27]:
scores = np.mean([cbm.predict_proba(test_pool)[:,1] for cbm in models],axis=0)

In [28]:
simple_sub['score'] = scores

In [29]:
simple_sub.to_csv('CatBoost_OnlyTSFRESH.csv',index=False)